In [1]:
import sys
import os

# Get the parent directory of the current file
parent_dir = os.path.abspath("..")  # Go up one level

# Add parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
import pandas as pd

from modules import cleaner

In [3]:
df = pd.read_csv("extracted.csv")

df_cleaned = cleaner.clean_and_format(df)
df_cleaned.head(10)

f:\Projects\Repos\solid_lux_repo\modules\cleaner.py:21: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min_idx = group[num_col].idxmin(skipna=True)
f:\Projects\Repos\solid_lux_repo\modules\cleaner.py:21: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min_idx = group[num_col].idxmin(skipna=True)
f:\Projects\Repos\solid_lux_repo\modules\cleaner.py:21: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min_idx = group[num_col].idxmin(skipna=True)
f:\Projects\Repos\solid_lux_repo\modules\cleaner.py:21: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min_idx = gro

,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,supp_fee_policy,coverage_BI,coverage_PD,coverage_MED,coverage_UM_UIMBI,coverage_UMPD,coverage_COMP,coverage_COLL,coverage_GAP
0,ACCC Insurance Company,NaN,None,1.0,The document shows multiple policy term option...,2.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
1,Access General Agency Insurance Agency of Texas,NaN,None,1.0,Document mentions one month policy in section ...,10.0,mandatory,mandatory,none,optional,optional,optional,optional,none
2,Accident Insurance Company,NaN,None,1.0,"Policies are written for One (1), Three (3) an...",1215.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
3,Aegis Security Insurance Company,90.0,The policy fee is fully earned at inception an...,6.0,Semi-annual (6 month) policy term is acceptabl...,5.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
4,Affirmative,NaN,None,NaN,None,NaN,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,none
5,Allied Property and Casualty Insurance Company,NaN,None,6.0,The document states that a Policy Period is si...,NaN,mandatory,mandatory,optional,optional,optional,optional,optional,optional
6,American Access Casualty Company,NaN,None,6.0,Based on the document title indicating effecti...,144.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
7,Auto Club County Mutual Insurance Company,NaN,None,12.0,The document mentions policy effective date wi...,NaN,mandatory,mandatory,optionnal,optionnal,optionnal,optionnal,optionnal,none
8,Bristol West Specialty,NaN,None,6.0,Policy terms available are 6 months and 12 mon...,NaN,mandatory,mandatory,optional,optional,optional,optional,optional,none
9,California Casualty Indemnity Exchange,NaN,None,6.0,Standard policy period appears to be 6 months ...,NaN,mandatory,mandatory,optional,optional,optional,optional,optional,none


In [4]:
# Save cleaned CSV file
cleaned_file_path = "./extracted_cleaned.csv"
df_cleaned.to_csv(cleaned_file_path, index=False)

# Show result preview
df_cleaned.head()

,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,supp_fee_policy,coverage_BI,coverage_PD,coverage_MED,coverage_UM_UIMBI,coverage_UMPD,coverage_COMP,coverage_COLL,coverage_GAP
0,ACCC Insurance Company,NaN,None,1.0,The document shows multiple policy term option...,2.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
1,Access General Agency Insurance Agency of Texas,NaN,None,1.0,Document mentions one month policy in section ...,10.0,mandatory,mandatory,none,optional,optional,optional,optional,none
2,Accident Insurance Company,NaN,None,1.0,"Policies are written for One (1), Three (3) an...",1215.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
3,Aegis Security Insurance Company,90.0,The policy fee is fully earned at inception an...,6.0,Semi-annual (6 month) policy term is acceptabl...,5.0,mandatory,mandatory,optional,optional,optional,optional,optional,none
4,Affirmative,NaN,None,NaN,None,NaN,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,mandatory,none


In [7]:
print(df_cleaned.to_markdown())

|    | company_name                                                      |   min_premium | min_premium_comments                                                                                                                                                                                                                                                     |   policy_period | policy_period_comments                                                                                                                                                                  |   supp_fee_policy | coverage_BI   | coverage_PD   | coverage_MED   | coverage_UM_UIMBI   | coverage_UMPD   | coverage_COMP   | coverage_COLL   | coverage_GAP   |
|---:|:------------------------------------------------------------------|--------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------